In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pywt
import seaborn as sns

sns.set_theme()
%matplotlib inline
mpl.rcParams['figure.dpi'] = 300

In [2]:
wavelet = pywt.Wavelet('db1')
dec_lo, dec_hi, rec_lo, rec_hi = wavelet.filter_bank

sg = np.arange(1,11)
padlen = len(dec_lo)
# print(f"padlen: {padlen}")
sig = np.pad(sg, padlen, mode='edge')
ca = np.convolve(sig, dec_lo, mode='full')[1 + padlen:-padlen:2]
cd = np.convolve(sig, dec_hi, mode='full')[1 + padlen:-padlen:2]
orig = pywt.dwt(sg, wavelet, mode='constant')
# print(f"Orig:{orig}")
# print(f"ca:{ca}")
# print(f"cd:{cd}")
# lev1 = pywt.idwt(cA=ca, cD=cd, wavelet=wavelet, mode='constant')

# print(f"l1:{lev1}")

# tst = np.pad(ca, padlen, mode='constant')
# tst = np.insert(ca, np.arange(len(ca)), ca)
# print(tst)
# rlo = np.convolve(tst, rec_lo, mode='full')
# rhi = np.convolve(tst, rec_hi, mode='full')
# print(f"rlo: {rlo}")
# print(f"rhi: {rhi}")
# print(rlo+rhi)
# print(f"sg:{sg}")
# print(pywt.waverec(coeffs=[ca,cd], wavelet=wavelet))

In [17]:
def dyadup(a, level=1, even=False):
    level += 1
    start = 1 if even else 0
    out = np.zeros(len(a) * level, dtype=a.dtype)
    out[start::level] = a
    return out

In [ ]:
def convolve_1d(s, kernel):
    rev_kernel = kernel[::-1].copy()
    padsize = kernel.size-1
    s = np.pad(s, padsize, mode='constant')
    n_steps = s.size - kernel.size + 1

    result = np.zeros(n_steps, dtype=np.double)
    n_ker = kernel.size
    for i in range(n_steps):
        result[i] = np.dot(s[i: i + n_ker], rev_kernel)
    return result

In [16]:
# print(np.convolve(sig, dec_lo, mode='full')[1 + padlen:-padlen:2])
# print(len(ca))
# print(len(dyadup(ca, even=True)))

cad = dyadup(ca, level=1, even=True)
cdd = dyadup(cd, level=1, even=True)

one = convolve_1d(cad, np.array(dec_lo))
two = convolve_1d(cdd, np.array(dec_hi))
# two = np.convolve(dyadup(ca), dec_hi, mode='full')
# print(one)
# print(two)
print(one-two)

In [ ]:
def reconstruct(coeffs: list, level: int, rec_high: np.array, rec_low: np.array) -> np.array:
    _ca = dyadup(coeffs[0], even=True)
    _cd1 = dyadup(coeffs[1], even=True)
    
    halfa = convolve_1d(_ca, np.array(dec_lo))
    halfb = convolve_1d(_cd1, np.array(dec_hi))
    
    sig = halfa-halfb
    
    return sig